In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_iris
from scipy.stats import multivariate_normal
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [2]:
def fit(x_train, y_train):
    m = y_train.shape[0]
    x_train = x_train.reshape(m, -1)
    input_feature = x_train.shape[1]
    class_label = len(np.unique(y_train.reshape(-1)))
    
    mu = np.zeros((class_label, input_feature))
    sigma = np.zeros((class_label, input_feature, input_feature))
    phi = np.zeros(class_label)

    for label in range(class_label):
        indices = (y_train == label)
        phi[label] = float(np.sum(indices)) / m
        mu[label] = np.mean(x_train[indices, :], axis=0)
        sigma[label] = np.cov(x_train[indices, :], rowvar=0)
    return phi, mu, sigma

In [3]:
def predict(x_tests, phi, mu, sigma):
    x_tests = x_tests.reshape(x_tests.shape[0], -1)
    class_label = mu.shape[0] 
    scores = np.zeros((x_tests.shape[0], class_label))  
    for label in range(class_label): 
        normal_distribution_prob = multivariate_normal(mean=mu[label], cov=sigma[label])
        for i, x_test in enumerate(x_tests):
            scores[i, label] = np.log(phi[label]) + normal_distribution_prob.logpdf(x_test)
    predictions = np.argmax(scores, axis=1)
    return predictions

In [4]:
data = load_iris()
x_train, x_test, y_train, y_test = train_test_split(data.data, data.target)
phi, mu, sigma = fit(x_train, y_train)
y_predict = predict(x_test, phi, mu, sigma)
score = f1_score(y_test, y_predict, average="weighted")
print("f1 score of our model: ", score)

f1 score of our model:  0.9204867006225242
